# **Questions for Assignment 2**

You have been given (on Moodle) a sample dataset of historical events, and a Spark program that implements a group by and count on a data set of historical events.

You can use this program as a template for the questions below.

Write a spark program that implements the following four queries as separate functions, and a main program that takes as argument an integer from 1 to 4, and executes the relevant query from the below:

> 1.Given an entity like Gandhi, Greece, etc. create a dataset of [entity, category1, category 2, count] considering events where entity occurs in the description





>  2.As above, given a file with entities, one per line.   Assume the entities are given in a file called entities.





> 3.Find the count of all instances in each  year (consider only events with granularity year)





> 4.For each year, find the rank of the year if sorted in descending order by number of events in the year.






In all cases the result should be computed using spark and output to console as in the template program.

In [2]:
!pip install pyspark==3.0.1 py4j==0.10.9 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 204.2 MB 31 kB/s 
     |████████████████████████████████| 198 kB 66.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=3d694f1a41270084c9b8b608d887ad7c0c1beb456e65c03b1efcf6765e638a9a
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

**Loading Data**


In [6]:
# Reading JSON file
json_file = '/history_english.json'
data = spark.read.json(json_file)
data.show(10)

+---------+---------------+----+--------------------+-----------+----+
|category1|      category2|date|         description|granularity|lang|
+---------+---------------+----+--------------------+-----------+----+
| By place|         Greece|-300|Pilgrims travel t...|       year|  en|
| By place|          Egypt|-300|Pyrrhus, the King...|       year|  en|
| By place|          Egypt|-300|Ptolemy concludes...|       year|  en|
| By place|Seleucid Empire|-300|Seleucus founds t...|       year|  en|
| By place|Seleucid Empire|-300|After the death o...|       year|  en|
| By place|          India|-300|The central texts...|       year|  en|
| By topic|            Art|-300|In Pella (in Mace...|       year|  en|
| By place| Roman Republic|-299|The Samnites, sei...|       year|  en|
| By place|          China|-299|The state of Qin ...|       year|  en|
| By place|          China|-299|King Wuling of Zh...|       year|  en|
+---------+---------------+----+--------------------+-----------+----+
only s

# **Question 1**

In [31]:
import pandas as pd
from pyspark.sql.functions import *

entity = 'Greece'

#removing rows which has category1==NULL and/or category2=NULL and taking this output in exdata for que1 and 2
data = data.where(data.category1.isNotNull())
data = data.where(data.category2.isNotNull())

exdata = data[data['description'].contains(entity)]   #finding entity in desciprion of data
exdata = exdata.withColumn('entity', lit(entity))       #adding entity column
exdata = exdata.groupby('entity','category1','category2').count()       #groupby and count

#storing output into question1-output.csv
exdata.toPandas().to_csv('question1-output.csv')

#print(exdata)
exdata.show(20)

+------+---------+--------------------+-----+
|entity|category1|           category2|count|
+------+---------+--------------------+-----+
|Greece| By place|      Roman Republic|   12|
|Greece| By place|          Asia Minor|    1|
|Greece| By place|              Greece|   24|
|Greece| By place|     Seleucid Empire|    4|
|Greece| By place|    Byzantine Empire|    4|
|Greece| By place|Eastern Roman Empire|    1|
|Greece| By place|        Roman Empire|   14|
|Greece| By place|              Europe|    5|
|Greece| By topic|            Religion|    2|
|Greece| By place|      =Mediterranean|    1|
+------+---------+--------------------+-----+



# **Question 2**

In [33]:
from pyspark.sql.functions import *
from pyspark.sql.functions import col, lit

with open("/entities.txt") as f:
    entities = f.readlines()

# new line characters removal
entities = [x.strip() for x in entities]
#print(entities)
 
finaldata = []
for entity in entities:
  exdata = data[data['description'].contains(entity)]
  exdata = exdata.withColumn('entity', lit(entity))
  exdata = exdata.groupby('entity','category1','category2').count()
  #exdata.show()
  exdata.toPandas().to_csv('question2-output.csv', mode='a', index=False, header=False)
  finaldata.append(exdata.show())

#storing output into question2_ut.csv
#finaldata.toPandas().to_csv('question2-output.csv')


print(finaldata)

+-------+---------+-----------------+-----+
| entity|category1|        category2|count|
+-------+---------+-----------------+-----+
|Ptolemy| By place|       Asia Minor|    3|
|Ptolemy| By place|         Anatolia|    1|
|Ptolemy| By topic|          Science|    1|
|Ptolemy| By place|  Seleucid Empire|   17|
|Ptolemy| By place|            Libya|    1|
|Ptolemy| By place|            Syria|    1|
|Ptolemy| By place|           Greece|   18|
|Ptolemy| By place|            Judea|    1|
|Ptolemy| By topic|Arts and sciences|    4|
|Ptolemy| By place|            Egypt|   70|
|Ptolemy| By place|   Roman Republic|    9|
+-------+---------+-----------------+-----+

+--------+---------+---------------+-----+
|  entity|category1|      category2|count|
+--------+---------+---------------+-----+
|Seleucus| By place|   Roman Empire|    2|
|Seleucus| By place|         Persia|    1|
|Seleucus| By place|         Greece|    9|
|Seleucus| By place|     Asia Minor|    6|
|Seleucus| By place|          Egypt|  

# **Question 3**

In [26]:
import pyspark
from pyspark.sql.functions import year
from pyspark.sql.functions import to_date
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
import pandas as pd


tempdata=[]                      #temporary list to store the data
for i in data.collect():
   tempdata.append(tuple(i))

dates = []                      #extracted the date
for i in range(len(tempdata)):
  tempdate = tempdata[i][2]
  dates.append(tempdate)

year = []                       #extracted the year from the dates
for i in range(len(dates)):
  date = dates[i]
  curryear = ''
  for j in range(len(date)):
    if(date[j]!='/'):
      curryear = curryear + date[j]
    else:
      break 
  year.append(curryear)

# appending this list of years to the original data

temp_year_df = pd.DataFrame(year)
tempdata_df = pd.DataFrame(tempdata)

tempdata_df['date'] = temp_year_df[0].values
#print(tempdata_df)

finaldata = tempdata_df.groupby('date')['date'].count().reset_index(name='counts')

#storing output into question3_output.csv
finaldata.to_csv('question3-output.csv')


finaldata

,date,counts
0,-1,4
1,-10,4
2,-100,13
3,-101,2
4,-102,3
...,...,...
1472,995,6
1473,996,9
1474,997,6
1475,998,4


# **Question 4**

In [28]:
#counts is the count of number events in the year (year is represented in date  column)
finaldata["Ranks"] = finaldata["counts"].rank()
finaldata.sort_values("counts", inplace = True, ascending=False)

#storing output into question4-output.csv
finaldata.to_csv('question4-output.csv')


finaldata

,date,counts,Ranks
386,1118,52,1477.0
399,1134,40,1476.0
299,1001,32,1475.0
374,1100,31,1474.0
298,1000,28,1473.0
...,...,...,...
615,210,1,27.5
582,183,1,27.5
60,-156,1,27.5
1257,799,1,27.5
